In [1]:
import ipyleaflet as ipl
import toml
import requests
import collections
import arrow

# See readme.md for credentials.toml formatting
cred = toml.load('credentials.toml')
appid = cred['api']['here']['appid']
appcode = cred['api']['here']['appcode']


def address_to_coord(address_string):
    """Return a tuple of the Lat/Long of the address."""
    raw_address = '+'.join(address_string.replace(',', ' ').split(' '))
    formatted_address = raw_address.replace('++', '+')

    url = "https://geocoder.api.here.com/6.2/geocode.json" +\
        "?app_id=" +\
        appid +\
        "&app_code=" +\
        appcode +\
        "&searchtext=" +\
        formatted_address

    response = requests.get(url)

    data = response.json()
    lat_long = data['Response']['View'][0]['Result'][0]['Location']['NavigationPosition'][0]
    point = coord_to_tuple(lat_long)
        
    return point


def get_isochrome(coordinates, timestamp, rangetype="time",
                  ranges=[600, 1200, 1800, 2400, 3000, 3600], points="2000"):
    """Format the Here.com API call, return the json response."""
    # Time ranges in seconds, Distance ranges in meters

    # Ensure ranges are strings instead of integers
    range_strings = list(map(str, ranges))
    ranges = ','.join(range_strings)
    
    # Ensure coordinates are a string
    coordinates = coord_to_string(coordinates)

    url = "https://isoline.route.cit.api.here.com/routing/" +\
        "7.2/calculateisoline.json?app_id=" +\
        appid +\
        "&app_code=" +\
        appcode +\
        "&mode=shortest;car;traffic:enabled&start=geo!" +\
        coordinates + "&maxpoints=" +\
        points + "&range=" +\
        ranges +\
        "&rangetype=" +\
        rangetype +\
        "&jsonAttributes=41"

    response = requests.get(url)

    data = response.json()
    
    return data


def array_to_points(points):
    """Transform an array of floats into a list of 2 tuples."""
    point_list = []

    array = collections.deque(points)
    while array:
        lat = array.popleft()
        long = array.popleft()
        point = (lat, long)
        point_list.append(point)

    return point_list


def drive_time_shapes(drive_time):
    """Simplify JSON response into a dictionary of point lists."""
    isochrones = {}

    for shape in drive_time['response']['isoline']:
        uid = str(int(shape['range'] / 60)) + ' minutes'

        points = shape['component'][0]['shape']

        point_list = array_to_points(points)

        isochrones[uid] = point_list

    return isochrones


def coord_to_tuple(coordinates):
    """Transform coordinates to a 2 tuple."""
    if type(coordinates) == str:
        center = tuple(map(float, coordinates.split(',')))

    elif type(coordinates) == dict:
        center = tuple(coordinates.values())

    elif type(coordinates) == tuple:
        center = coordinates

    elif (type(coordinates) == list) and (len(coordinates) == 2):
        center = tuple(coordinates)

    else:
        print('invalid coordinate type')
        center = None

    return center

def coord_to_string(coordinates):
    """Transform coordinates into a string for API"""
    if type(coordinates) == tuple:
        string = ','.join(list(map(str, coordinates)))
        
    elif type(coordinates) == dict:
        string = ','.join(list(map(str, coordinates.values())))
    
    elif (type(coordinates) == list) and (len(coordinates) == 2):
        string = ','.join(list(map(str, coordinates)))
        
    elif type(coordinates) == str:
        string = coordinates
        
    else:
        print('invalid coordinate type')
        string = None        

    return string

def plot_isochrones(coordinates, isochrones):
    '''Initiate map and plot isochrones centered on coordinates.'''

    center = coord_to_tuple(coordinates)

    layer_color = 'brown'
    fill_color = 'orange'
    m = ipl.Map(center=center, zoom=8)
    for key in isochrones.keys():
        print(key)
        item = ipl.Polygon(
            locations=isochrones[key],
            color=layer_color,
            weight=1,
            fill_color=fill_color,
            fill_opacity=0.2,
            name=key)
        m.add_layer(item)

    m.add_control(ipl.LayersControl())

    return m


def main(coordinates=None, timestamp=None, address=None):

    if not timestamp:
        timestamp = str(arrow.now().floor('second')).split('+')[0] + 'Z'
    
    if address:
        coordinates = address_to_coord(address)
        
    elif coordinates:
        coordinates = coord_to_string(coordinates)
    
    else:
        print('Please include valid address or coordinates (Latitude, Longitude)')
        
        return None
    
    drive_time = get_isochrome(coordinates, timestamp)

    isochrones = drive_time_shapes(drive_time)

    m = plot_isochrones(coordinates, isochrones)
    
    return m




In [2]:
# Address for Red Hat Tower
address = "190 E. Davie Street, Raleigh, NC 27601"

m = main(address=address)
m

10 minutes
20 minutes
30 minutes
40 minutes
50 minutes
60 minutes


Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …